In [566]:
import openai
import langchain
import httpx
import langgraph
import dotenv
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from typing import List
from pydantic import BaseModel, Field

In [567]:
load_dotenv()

True

In [568]:
# Access the variables
# openai_api_key = os.getenv('OPENAI_API_KEY')
JINA_API_KEY = os.getenv('JINA_API_KEY')
app_password = os.getenv('APP_PASSWORD')
# email = os.getenv('EMAIL')

client = OpenAI(
    api_key="your_api_key",
)

In [569]:
# Building Agents without Langchain

def run_completion(system_prompt, user_prompt, model):
  completion = client.beta.chat.completions.parse(
      model="gpt-4o-mini",
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt},
      ],
    response_format=model,
  )
  return completion.choices[0].message.parsed

In [570]:
from pydantic import BaseModel

class Plan(BaseModel):
  """Use to write a plan of actions to take for given task."""
  actions: List[str] = Field(description="List of actions to take")

In [571]:
emails = [

]


In [572]:
system_prompt = """
You are an expert at writing newsletters, and your task is to use the given context to write email newsletters and send it to appropriate recipients

These tools are available to you:

- scrape_website: Use to get the text from a website
- write_newsletter: Use to write a newsletter from the provided text
- send_email: Sends email to multiple users
"""

In [573]:
actions = run_completion("", f"Write a newsletter using text from https://paulgraham.com/startupideas.html and send it to the following emails {emails}", Plan)

In [574]:
for action in actions.actions:
  print(action)

Visit the specified URL: https://paulgraham.com/startupideas.html
Read and summarize the key points from the webpage
Compose a newsletter incorporating the key points
Format the newsletter for clarity and readability
Include a subject line for the email
Prepare the email for sending
Add the recipient email addresses (which were supposed to be provided)
Review the email for any errors or necessary adjustments
Send the email to all specified addresses


In [575]:
emails = []

In [576]:
steps_to_take = run_completion(system_prompt, f"Write a newsletter using text from https://paulgraham.com/startupideas.html and send it to the following emails {emails}", Plan)

In [577]:
for action in steps_to_take.actions:
  print(action)

Scrape the website https://paulgraham.com/startupideas.html to gather the content for the newsletter.
Organize and summarize the scraped content to create a concise and engaging newsletter.
Use the write_newsletter tool to format the newsletter properly based on the summarised content.
Send the newsletter to the specified email addresses using the send_email tool.


In [578]:
messages = []

In [579]:
class SendEmail(BaseModel):
  """Use to send email to multiple users"""
  subject: str
  body: str


class Newsletter(BaseModel):
  """Use to write a newsletter from the provided text"""
  text: str


class ScrapeWebsite(BaseModel):
  """Use to obtain text from a given url"""
  url: str

In [580]:
def send_email(subject: str, body: str):
  """Sends email to multiple users

  Args:
    subject: subject of email
    body: body of the email
    recipients: list of email addresses
  """
  emails = [
    "hamzatakolade@gmail.com",
    "hamzatakolade111@gmail.com",
    "hamzatakolade84@gmail.com",
    "emmanuelchijiaku3@gmail.com"
  ]
  msg = MIMEText(body)
  msg['Subject'] = subject
  msg['From'] = sender
  msg['To'] = ', '.join(emails)
  with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
      smtp_server.login(sender, password)
      smtp_server.sendmail(sender, emails, msg.as_string())
  return "Message sent!"

In [581]:
def write_newsletter(text: str):
  """
  Use to write a newsletter from the provided text

  Args:
    - text: text to write the newsletter from
  """
  completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You're an expert at writing email newsletters. Use the provided context to write a newsletter to a general audience."},
        {"role": "user", "content": f"Context: {text}"},
    ],
   response_format=Email,
)
  return json.dumps(completion.choices[0].message.parsed.dict())

In [582]:
def scrape_website(url: str):
  """
  Use to get the text from a website

  Args:
    - url: url to scrape
  """
  headers = {'Authorization': f'Bearer {JINA_API_KEY}'}

  response = httpx.get(f'https://r.jina.ai/{url}', headers=headers)

  return response.text

In [583]:
tools = [openai.pydantic_function_tool(SendEmail), openai.pydantic_function_tool(Newsletter), openai.pydantic_function_tool(ScrapeWebsite)]

In [584]:
def completion(messages, tools):
  response = client.beta.chat.completions.parse(
      model="gpt-4o-mini",
      messages=messages,
      tools=tools,
  )

  return response.choices[0].message

In [585]:
steps_to_take.actions[0]

'Scrape the website https://paulgraham.com/startupideas.html to gather the content for the newsletter.'

In [586]:
res = completion([{"role": "user", "content": steps_to_take.actions[0]}], tools)

In [587]:
res

ParsedChatCompletionMessage[NoneType](content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ParsedFunctionToolCall(id='call_XjMk5enA2Taq60ZAsgMSyhCz', function=ParsedFunction(arguments='{"url":"https://paulgraham.com/startupideas.html"}', name='ScrapeWebsite', parsed_arguments=ScrapeWebsite(url='https://paulgraham.com/startupideas.html')), type='function')], parsed=None)

In [588]:
res.tool_calls[0]

ParsedFunctionToolCall(id='call_XjMk5enA2Taq60ZAsgMSyhCz', function=ParsedFunction(arguments='{"url":"https://paulgraham.com/startupideas.html"}', name='ScrapeWebsite', parsed_arguments=ScrapeWebsite(url='https://paulgraham.com/startupideas.html')), type='function')

In [589]:
res.tool_calls[0].id

'call_XjMk5enA2Taq60ZAsgMSyhCz'

In [590]:
res.tool_calls[0].function

ParsedFunction(arguments='{"url":"https://paulgraham.com/startupideas.html"}', name='ScrapeWebsite', parsed_arguments=ScrapeWebsite(url='https://paulgraham.com/startupideas.html'))

In [591]:
def map_function_call(arguments):
  function_map = {
      "ScrapeWebsite": scrape_website,
      "Newsletter": write_newsletter,
      "SendEmail": send_email
  }

  function_name = arguments.tool_calls[0].function.name
  func_args = json.loads(arguments.tool_calls[0].function.arguments)

  return function_map[function_name](**func_args)

In [592]:
map_function_call(res)

'Title: How to Get Startup Ideas\n\nURL Source: https://paulgraham.com/startupideas.html\n\nMarkdown Content:\n| ![Image 1: How to Get Startup Ideas](https://s.turbifycdn.com/aah/paulgraham/how-to-get-startup-ideas-2.gif)\n\nNovember 2012\n\nThe way to get startup ideas is not to try to think of startup ideas. It\'s to look for problems, preferably problems you have yourself.\n\nThe very best startup ideas tend to have three things in common: they\'re something the founders themselves want, that they themselves can build, and that few others realize are worth doing. Microsoft, Apple, Yahoo, Google, and Facebook all began this way.\n\n**Problems**\n\nWhy is it so important to work on a problem you have? Among other things, it ensures the problem really exists. It sounds obvious to say you should only work on problems that exist. And yet by far the most common mistake startups make is to solve problems no one has.\n\nI made it myself. In 1995 I started a company to put art galleries onli

In [593]:
function_call_result_message = {
    "role": "tool",
    "content": json.dumps({
        "order_id": order_id,
        "delivery_date": delivery_date.strftime('%Y-%m-%d %H:%M:%S')
    }),
    "tool_call_id": response['choices'][0]['message']['tool_calls'][0]['id']
}

NameError: name 'order_id' is not defined

In [594]:
def workflow(messages=[]):
  for action in steps_to_take.actions:
    messages.append({"role": "user", "content": action})
    ai_msg = completion(messages, tools)
    messages.append(ai_msg)
    function_response = map_function_call(ai_msg.tool_calls[0].function)
    messages.append({"role": "tool", "name": ai_msg.tool_calls[0].function.name, "content": function_response, "tool_call_id": ai_msg.tool_calls[0].id})

In [595]:
workflow()

AttributeError: 'ParsedFunction' object has no attribute 'tool_calls'

In [ ]:
messages